Part 4 of the LLM per game TLDR generation project

some of the game contains the topic stuff, some of the game does not

assume we target with games with custom topics

In [2]:
import pandas as pd
import numpy as np

from pathlib import Path
import os, sys, pickle, json, traceback
from datetime import datetime

In [3]:
game_steamid = 1716740              # starfield
game_name = 'starfield'             # also the folder name where the reviews are stored

# game_steamid = 1118010
# game_name = 'monster_hunter_world_iceborne'

# game_steamid = 582010
# game_name = 'monster_hunter_world'

# game_steamid = 2138330          # cyberpunk2077 phantom liberty
# game_name = 'cyberpunk2077_phantom_liberty'

# game_steamid = 1091500          # cyberpunk2077
# game_name = 'cyberpunk2077'

# game_steamid = 730
# game_name = 'counter-strike_2'

# game_steamid = 570
# game_name = "dota2"


In [6]:
# load the reviews from folder

reviews_reqs = []

# get existing folder and retrieve the cursor object (?)

# load the latest file
game_folder = Path(f'../../dataset/data_scraping/steam_comments_scraping/{game_name}')
if game_folder.exists():
    try:
        latest_file_path = game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis.pkl')
        with open(latest_file_path, 'rb') as f:
            reviews_reqs = pickle.load(f)           # retrieve the list of reviews
            print('Loaded:', latest_file_path)
    except IndexError as e:
        print('Error loading the latest file:', e)
        traceback.print_exc()

print()

# also load different generated data
ageReviews_df = pd.read_csv(game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_ageGroup.csv'), index_col=None)
print('Loaded ageReviews_df')
genderReviews_df = pd.read_csv(game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_genderReviews.csv'), index_col=None)
print('Loaded genderReviews_df')
sentimentReviews_df = pd.read_csv(game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_sentimentReviews.csv'), index_col=None)
print('Loaded sentimentReviews_df')
sentimentReviews_truelabel_df = pd.read_csv(game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_sentimentReviews_truelabel.csv'), index_col=None)
print('Loaded sentimentReviews_truelabel_df')
sentimentByAgeGroup_df = pd.read_csv(game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_sentimentByAgeGroup.csv'), index_col=None)
print('Loaded sentimentByAgeGroup_df')
sentimentByGender_df = pd.read_csv(game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_sentimentByGender.csv'), index_col=None)
print('Loaded sentimentByGender_df')

Loaded: ../../dataset/data_scraping/steam_comments_scraping/starfield/steam_reviews_1716740_unique_with_gendata_with_analysis.pkl

Loaded ageReviews_df
Loaded genderReviews_df
Loaded sentimentReviews_df
Loaded sentimentReviews_truelabel_df
Loaded sentimentByAgeGroup_df
Loaded sentimentByGender_df


In [19]:
ageReviews_df

,age_group,count
0,13-19,9833
1,20-29,38403
2,30-39,29110
3,40-49,14313
4,50-59,4800
5,60-69,0
6,70-79,0
7,80-89,0
8,90-99,0
9,NaN,0


In [8]:
from langchain_community.llms import Ollama
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
# from langchain.vectorstores import Chroma

/root/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# prompt constants

SYSTEM_PROMPT_TEMPLATE = \
'''You have access to a pandas dataframe `df`. {description_of_the_table}
Here is the output of `df.to_markdown()`:
```
{df_markdown}
```
'''

# TODO: can further optimize?? Currently this is copied from IELTS academic writing task 1
PROMPT_TEMPLATE = \
'''Summarise the information by selecting and reporting the main features, and main comparisons where relevant. Output a paragraph with less than 50 words.
Only output the paragraph. Do NOT output other text.'''


DESCRIPTION_AGEREVIEW_DF = \
'''df is a dataframe that describes the distribution of reviews by age group.'''

In [10]:
llm_mistral7b = Ollama(
    model='mistral:7b-instruct-v0.2-q4_0', temperature=0.4,
    num_gpu = 1,        # disable/enable gpu for testing
)

In [15]:
# define callbacks for detecting token usage (Ollama only)
# copoed from llm_rag_comparison.ipynb

from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.outputs.llm_result import LLMResult
from collections import deque

class TokenUsageCallbackHandler(BaseCallbackHandler):

    def __init__(self, deque: deque = None):
        super().__init__()
        self.deque = deque

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        # print(response)

        generation = response.generations[0][0]
        gen_info = generation.generation_info

        # get token usage
        # ref: https://github.com/orgs/langfuse/discussions/1179
        token_usage = gen_info.get('prompt_eval_count', 0) + gen_info.get('eval_count', 0)
        # get time costed (local machine)
        # instead of getting total duration, we get the prompt_eval_duration and eval_duration to exclude the load duration (e.g. to load the model to the gpu, etc.)
        time_costed = gen_info.get('prompt_eval_duration', 1e-10) + gen_info.get('eval_duration', 1e-10)     # in ns, a small value to indicate a inf time when it fails


        # create an object to store the token usage and time costed
        token_usage_obj = {
            'token_usage': token_usage,
            'time_costed': time_costed
        }

        # append the object to the deque
        self.deque.append(token_usage_obj)


def combine_token_usage_obj(token_usage_obj_list:list[dict]) -> dict:
    token_usage = 0
    time_costed = 0
    for obj in token_usage_obj_list:
        token_usage += obj['token_usage']
        time_costed += obj['time_costed']

    return {
        'token_usage': token_usage,
        'time_costed': time_costed
    }



common_deque = deque()
chain_config = {
    "callbacks": [TokenUsageCallbackHandler(common_deque)],
}

In [17]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

chat_prompt_01 = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT_TEMPLATE),
    ("user", PROMPT_TEMPLATE),
])

chain_01 = chat_prompt_01 | llm_mistral7b

resp_01 = chain_01.invoke({
    'df_markdown': ageReviews_df.to_markdown(),
    'description_of_the_table': DESCRIPTION_AGEREVIEW_DF,
}, config=chain_config)

token_usage_01 = common_deque.popleft()

print(resp_01)
    

 The dataframe `df` presents age group distributions of reviews, with the largest number of reviews falling in the 20-29 age group (38,403). The youngest age group, 13-19, has 9,833 reviews, while other age groups, from 30-59, have substantial numbers as well. No reviews were recorded for ages 60 and above.
